# 屏幕截图

In [ ]:
`pip install mss numpy

In [ ]:
# adam算法测截图速度
import multiprocessing
import mss
import time
import os
import numpy as np

global max_count
global monitor_num
global fr
global grad_shoot_count
global grad_count
global shoot_count
global shoot_last_time
global m
global m_
global v
global v_
global beta1
global beta2
global lr
global sigma

max_count = 12000 # 最大张数
monitor_num = 1 # 显示器编号
fr = 60.0 # 初始帧率
grad_shoot_count = 0.0
grad_count = 0
shoot_count = 0
shoot_last_time = time.time()
# adam参数
m = 0
m_ = 0
v = 0
v_ = 0
beta1 = 0.99
beta2 = 0.99
lr = 0.1
sigma = 0.01

def renew_grad(_value):
    global shoot_count
    shoot_count += 1
    current = time.time()
    global grad_shoot_count
    global shoot_last_time
    grad_shoot_count = 1 / (current - shoot_last_time + sigma)
    shoot_last_time = current
    

def take_screenshot(count, num = 1):
    with mss.mss() as sct:
        monitor = sct.monitors[num]
        return count, sct.grab(monitor)

def check_time():
    count = 0
    screenshot_pool = multiprocessing.Pool(os.cpu_count()*2-1) # 来个进程池
    ori_time = time.time()
    global shoot_last_time
    shoot_last_time = time.time()
    last_time = time.time()
    while True:
        current = time.time()
        global grad_count
        grad_count = 1/(current - last_time + sigma)
        last_time = current
        if count > max_count:
            break
        count += 1
        # print('count', count)
        screenshot_pool.apply_async(take_screenshot, args=(count, monitor_num, ), callback=renew_grad)
        grad = grad_shoot_count-grad_count
        global m
        global v
        global m_
        global v_
        m_ = beta1*m + (1-beta1)*grad
        v_ = beta2*v + (1-beta2)*grad**2
        m__ = m_/(1-beta1**count)
        v__ = v_/(1-beta2**count)
        global fr
        fr = fr + lr*m__/(np.sqrt(v__)+sigma)
        print(fr, '--- fr update ---', grad)
        m = m_
        v = v_
        wait_time = count / fr + ori_time - time.time()
        wait_time = wait_time if wait_time > 0 else 0
        time.sleep(wait_time) # 按当前帧率等待
    screenshot_pool.close()

if __name__ == "__main__":
    check_time()

In [ ]:
# 多线程截图器
import time
import mss
import multiprocessing

class process_manage(): # 多线程基类
    def __init__(self):
        process = multiprocessing.Process(target=self.start)
        process.start()

class timer(process_manage): # 计时器
    def __init__(self, fps, target, max):
        self.fps = fps
        self.target = target
        self.max = max
        super().__init__()

    def start(self):
        ori_time = time.time()
        for index in range(self.max):
            print(index)
            self.target.send(index)
            wait_time = index / self.fps + ori_time - time.time()
            wait_time = max(wait_time, 0.0)
            time.sleep(wait_time)
        self.target.close()

class task_scheduler(process_manage): # 任务分发
    def __init__(self, src, tar_group):
        self.src = src
        self.tar_group = tar_group
        super().__init__()

    def start(self):
        current = 0
        try:
            while True:
                count = self.src.recv()
                self.tar_group[current % len(self.tar_group)].send(count)
                current += 1
        except EOFError:
            for target in self.tar_group:
                target.close()
            print('task done')

class screenshot_process(process_manage): # 截图器，会产生多个实例并行截图
    def __init__(self, source, target, monitor_num):
        self.source = source
        self.target = target
        self.monitor_num = monitor_num
        super().__init__()

    def start(self):
        try:
            with mss.mss() as sct:
                while True:
                    monitor = sct.monitors[self.monitor_num]
                    count = self.source.recv()
                    sct.grab(monitor)
                    print('shot', count)
        except EOFError:
            print('screenshot done')

if __name__ == "__main__":
    fps = 60.0
    max_count = 120
    monitor_num = 1
    send = []
    receive = []
    for i in range(11):
        tem_send, tem_receive = multiprocessing.Pipe()
        send.append(tem_send)
        receive.append(tem_receive)
    receive_group = []
    for i in range(10):
        screenshot_process(receive[i + 1], None, monitor_num)
    route = task_scheduler(receive[0], send[1:])
    timer_ = timer(fps, send[0], max=max_count)
    for item in send:
        item.close() # 主进程和子进程中的通信管道都要close掉才能引发EOF

有需求建议直接使用ffmpeg